In [1]:
from qiskit_emulator import EmulatorProvider
from qiskit import QuantumCircuit

provider = EmulatorProvider()

In [2]:
RUNTIME_PROGRAM = """
# This code is part of qiskit-runtime.
#
# (C) Copyright IBM 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

# This is a simplified version of the circuit-runner program.

from qiskit.compiler import transpile, schedule


def main(
    backend,
    user_messenger,
    circuits,
    initial_layout=None,
    seed_transpiler=None,
    optimization_level=None,
    transpiler_options=None,
    scheduling_method=None,
    schedule_circuit=False,
    inst_map=None,
    meas_map=None,
    measurement_error_mitigation=False,
    **kwargs,
):

    # transpiling the circuits using given transpile options
    transpiler_options = transpiler_options or {}
    circuits = transpile(
        circuits,
        initial_layout=initial_layout,
        seed_transpiler=seed_transpiler,
        optimization_level=optimization_level,
        backend=backend,
        **transpiler_options,
    )

    if schedule_circuit:
        circuits = schedule(
            circuits=circuits,
            backend=backend,
            inst_map=inst_map,
            meas_map=meas_map,
            method=scheduling_method,
        )

    if not isinstance(circuits, list):
        circuits = [circuits]

    # Compute raw results
    result = backend.run(circuits, **kwargs).result()

    if measurement_error_mitigation:
        # Performs measurement error mitigation.
        pass

    user_messenger.publish(result.to_dict(), final=True)
"""

In [3]:
RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

In [4]:
program_id = provider.runtime.upload_program(RUNTIME_PROGRAM, metadata=RUNTIME_PROGRAM_METADATA)

In [5]:
N = 6
qc = QuantumCircuit(N)

qc.x(range(0, N))
qc.h(range(0, N))

for kk in range(N//2,0,-1):
    qc.ch(kk, kk-1)
for kk in range(N//2, N-1):
    qc.ch(kk, kk+1)
qc.measure_all()

In [6]:
program_inputs = {
    'circuits': qc,
    'shots': 2048,
    'optimization_level': 0,
    'initial_layout': [0,1,4,7,10,12],
    'measurement_error_mitigation': False
}

In [7]:
runtime_program = provider.runtime.program(program_id)

In [8]:
job = provider.runtime.run(program_id, options=None, inputs=program_inputs)

DEBUG:qiskit_emulator.emulation_executor:starting ['/root/miniconda3/envs/qruntime/bin/python', '/tmp/tmp087w72ac/executor.py']


MESSENGER RECEIVED: {'backend_name': 'aer_simulator', 'backend_version': '0.8.2', 'qobj_id': '0c47feec-2880-4b5f-825c-f710b095804c', 'job_id': 'f5e8fb96-9e86-42c6-ac5a-2c57330afc41', 'success': True, 'results': [{'shots': 2048, 'success': True, 'data': {'counts': {'0x21': 35, '0x20': 34, '0x31': 66, '0x37': 229, '0x1': 38, '0x23': 55, '0x3f': 1020, '0x30': 59, '0x33': 139, '0x0': 38, '0x27': 130, '0x3': 73, '0x7': 132}}, 'meas_level': 2, 'header': {'clbit_labels': [['meas', 0], ['meas', 1], ['meas', 2], ['meas', 3], ['meas', 4], ['meas', 5]], 'creg_sizes': [['meas', 6]], 'global_phase': 0.0, 'memory_slots': 6, 'metadata': None, 'n_qubits': 13, 'name': 'circuit-0', 'qreg_sizes': [['q', 13]], 'qubit_labels': [['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4], ['q', 5], ['q', 6], ['q', 7], ['q', 8], ['q', 9], ['q', 10], ['q', 11], ['q', 12]]}, 'status': 'DONE', 'seed_simulator': 3210983101, 'metadata': {'parallel_state_update': 32, 'measure_sampling': True, 'method': 'statevector', 'parall

DEBUG:qiskit_emulator.emulation_executor:finished executing ['/root/miniconda3/envs/qruntime/bin/python', '/tmp/tmp087w72ac/executor.py']
DEBUG:qiskit_emulator.emulation_executor:stdout: exit

DEBUG:qiskit_emulator.emulation_executor:stderr: 
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/root/miniconda3/envs/qruntime/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/root/miniconda3/envs/qruntime/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/root/miniconda3/envs/qruntime/lib/python3.9/site-packages/qiskit_emulator/emulator_runtime_job.py", line 78, in poll_for_results
    response = requests.get(url)
  File "/root/miniconda3/envs/qruntime/lib/python3.9/site-packages/requests/api.py", line 76, in get
    return request('get', url, params=params, **kwargs)
  File "/root/miniconda3/envs/qruntime/lib/python3.9/site-packages/requests/api.py", line 61, in request
    return sess